<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/01_Data_collection_Country_P17_P27_P495.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import of the training data

from google.colab import files

# Import of moduls

!pip install wikidata --quiet

import matplotlib.pyplot as plt
import pandas as pd
from wikidata.client import Client

from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/France_Poland_multilanguage_shared_folder/[MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv',sep='\t')

Mounted at /content/drive


# Training dataset

In [ ]:
def extract_entity_id(url):
    return url.strip().split("/")[-1]

In [ ]:
def get_claims(wikidata_url):
  # Extract the id of the item
    entity_id = extract_entity_id(wikidata_url)

    # Extract information of the item
    client = Client()
    item = client.get(entity_id, load=True)

    # Extract the claims of the item
    claims = item.data.get("claims", {})

    return claims

In [ ]:
def get_country(wikidata_url):

    claims = get_claims(wikidata_url)
    client = Client()

    country_of_origin_id = None
    country_id = None
    nationality_id = None

    # Extract the statement "P495" : Country of origin
    unicode_statements = claims.get("P495", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})

            country_of_origin_id = value.get("id")

            return country_of_origin_id, None, None


    # Extract the statement "P17" : Country
    unicode_statements = claims.get("P17", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})
            country_id = value.get("id")

            return None, country_id, None


    # Extract the statement "P27" : Nationality
    unicode_statements = claims.get("P27", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})
            nationality_id = value.get("id")

            return None, None, nationality_id

    return country_of_origin_id, country_id, nationality_id

In [ ]:
# Due to some errors when we apply get_country to the dataframe, we create a more safety function

def safe_get_country(wikidata_url):
    try:
        return get_country(wikidata_url)
    except:
        return None, None, None

In [ ]:
# We merge the variables, in order to have only one.

def merge_variables(df, variables):
  """
  Take in input, the variables of a dataframe, and return a dataframe with a single variable => the first value founded across the rows
  """
  return df[variables].bfill(axis=1).iloc[:, 0]

In [ ]:
# Application to the whole dataset
train_df[['country_of_origin', 'country', 'nationality']] = train_df['item'].apply(lambda url: pd.Series(safe_get_country(url)))

In [ ]:
train_df

,item,name,description,type,category,subcategory,label,country_of_origin,country,nationality
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,None,None,Q298
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,None,None,Q153136
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,None,None,Q1747689
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,None,None,Q1041
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,None,None,Q30
...,...,...,...,...,...,...,...,...,...,...
6246,http://www.wikidata.org/entity/Q321103,Bühl,"quarter of Tübingen, Baden-Württemberg, Germany",entity,geography,neighborhood,cultural exclusive,None,Q183,None
6247,http://www.wikidata.org/entity/Q338167,Tenderloin,area of New York City during the late 19th and...,entity,geography,neighborhood,cultural exclusive,None,Q30,None
6248,http://www.wikidata.org/entity/Q66991,Schinznach-Dorf,former municipality and current district of Sc...,entity,geography,neighborhood,cultural exclusive,None,Q39,None
6249,http://www.wikidata.org/entity/Q66922,Ependes,village and former municipality in Bois-d'Amon...,entity,geography,neighborhood,cultural exclusive,None,Q39,None


In [ ]:
# Merge the 3 features
train_df['main_country'] = merge_variables(train_df, ['country_of_origin', 'country', 'nationality'])

# Delete the 3 other features
train_df.drop(['country_of_origin', 'country', 'nationality'], axis=1, inplace=True)

In [ ]:
# Visualisation
train_df

,item,name,description,type,category,subcategory,label,main_country
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,Q298
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,Q153136
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,Q1747689
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,Q1041
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,Q30
...,...,...,...,...,...,...,...,...
6246,http://www.wikidata.org/entity/Q321103,Bühl,"quarter of Tübingen, Baden-Württemberg, Germany",entity,geography,neighborhood,cultural exclusive,Q183
6247,http://www.wikidata.org/entity/Q338167,Tenderloin,area of New York City during the late 19th and...,entity,geography,neighborhood,cultural exclusive,Q30
6248,http://www.wikidata.org/entity/Q66991,Schinznach-Dorf,former municipality and current district of Sc...,entity,geography,neighborhood,cultural exclusive,Q39
6249,http://www.wikidata.org/entity/Q66922,Ependes,village and former municipality in Bois-d'Amon...,entity,geography,neighborhood,cultural exclusive,Q39


In [ ]:
# Exportation

train_df.to_json("train_df_country.json", orient="records", lines=True)
files.download('train_df_country.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Dev dataset

In [2]:
dev_df = pd.read_csv('/content/drive/MyDrive/France_Poland_multilanguage_shared_folder/validation_df.csv', index_col=0)

In [ ]:
# Application to the whole dataset
dev_df[['country_of_origin', 'country', 'nationality']] = dev_df['item'].apply(lambda url: pd.Series(safe_get_country(url)))

In [ ]:
# Merge the 3 features
dev_df['main_country'] = merge_variables(dev_df, ['country_of_origin', 'country', 'nationality'])

# Delete the 3 other features
dev_df.drop(['country_of_origin', 'country', 'nationality'], axis=1, inplace=True)

In [ ]:
# Visualisation
dev_df

,item,name,description,type,category,subcategory,label,main_country
0,http://www.wikidata.org/entity/Q15786,1. FC Nürnberg,"German sports club based in Nuremberg, Bavaria",entity,sports,sports club,cultural representative,Q183
1,http://www.wikidata.org/entity/Q268530,77 Records,UK record label,entity,music,record label,cultural exclusive,Q145
2,http://www.wikidata.org/entity/Q216153,A Bug's Life,1998 animated film directed by John Lasseter a...,entity,comics and anime,animated film,cultural representative,Q30
3,http://www.wikidata.org/entity/Q593,A Gang Story,2011 film by Olivier Marchal,entity,films,film,cultural exclusive,Q142
4,http://www.wikidata.org/entity/Q192185,Aaron Copland,"American composer, composition teacher, writer...",entity,performing arts,choreographer,cultural representative,Q30
...,...,...,...,...,...,...,...,...
295,http://www.wikidata.org/entity/Q36180,writer,person who uses written words to communicate i...,concept,visual arts,artist,cultural agnostic,None
296,http://www.wikidata.org/entity/Q156316,Würzburg Residence,cultural heritage monument D-6-63-000-454 (0) ...,entity,history,museum,cultural representative,Q183
297,http://www.wikidata.org/entity/Q56911,xunfu,high government official of the Chinese Ming a...,concept,politics,politician,cultural exclusive,Q9903
298,http://www.wikidata.org/entity/Q377,Yanka Kupala,Belarusian writer (1882–1942),entity,literature,poet,cultural representative,Q34266


In [ ]:
# Exportation

dev_df.to_json("dev_df_country.json", orient="records", lines=True)
files.download('dev_df_country.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>